<a href="https://colab.research.google.com/github/brendanlooker/colab-examples/blob/main/dataform/Simple_Composer_Dataform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sample Code to execute a Dataform pipeline

from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.dummy import DummyOperator
from airflow.utils.dates import days_ago
import logging

from airflow.providers.google.cloud.operators.dataform import (
    DataformCancelWorkflowInvocationOperator,
    DataformCreateCompilationResultOperator,
    DataformCreateWorkflowInvocationOperator,
    DataformGetCompilationResultOperator,
    DataformGetWorkflowInvocationOperator,
)

# Set up logging
logging.basicConfig(level=logging.INFO)

PROJECT_ID = ''
REGION = 'europe-west4'
REPOSITORY_ID = 'dataform-demo'
GIT_COMMITISH = 'main'


default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'start_date':  days_ago(1),
    'retry_delay': timedelta(minutes=30),
}
with DAG(
    'run_data_pipeline',
    default_args=default_args,
    description='Execute Data Pipeline',
    schedule_interval=timedelta(days=1),
) as dag:


    # Create Dataform compilation result
    create_dataform_compilation_result = DataformCreateCompilationResultOperator(
        task_id="create_df_compilation_result",
        project_id=PROJECT_ID,
        region=REGION,
        repository_id=REPOSITORY_ID,
        compilation_result={
            "git_commitish": GIT_COMMITISH,
        },
    )

    # Run Dataform job
    create_dataform_workflow_invocation = DataformCreateWorkflowInvocationOperator(
        task_id='create_dataform_workflow_invocation',
        project_id=PROJECT_ID,
        region=REGION,
        repository_id=REPOSITORY_ID,
            workflow_invocation={
            "compilation_result": "{{ task_instance.xcom_pull('create_df_compilation_result')['name'] }}",
            "invocation_config": {
                "included_tags": ["local"],
            },
        },
    )

    finish_pipeline = DummyOperator(
        task_id = 'finish_pipeline',
        dag = dag
    )

    # Set task dependencies
    create_dataform_compilation_result >> create_dataform_workflow_invocation >> finish_pipeline
